<a href="https://colab.research.google.com/github/slazur83/Tableau/blob/main/Weight_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [205]:
import pandas as pd
import glob
import os
import json
import numpy as np
from datetime import datetime
from google.colab import userdata
from google.colab import files

In [206]:
config_file_path = "/content/drive/MyDrive/Colab Notebooks/config.json"

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

google_sheets_moja_waga = config['google_sheets_moja_waga']
document_key = google_sheets_moja_waga.split('/')[-2]

**Google Sheets**

In [215]:
# data from 2020
google_sheets_moja_waga_2020 = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&gid=0'
df1 = pd.read_csv(google_sheets_moja_waga_2020, header=1)

In [216]:
# data from 2017
google_sheets_moja_waga_2017 = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&gid=2116961746'
df2 = pd.read_csv(google_sheets_moja_waga_2017, header=1)
df2['data'] = pd.to_datetime(df2['data'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
df2['waga [kg]'] = df2['waga [kg]'].str.replace(' kg', '').astype(float)

In [217]:
# data from 2016
google_sheets_moja_waga_2016 = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv&gid=552331991'
df3 = pd.read_csv(google_sheets_moja_waga_2016, header=1)
df3['data'] = pd.to_datetime(df3['data'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
df3['waga [kg]'] = df3['waga [kg]'].str.replace(' kg', '').str.strip().astype(float)

In [218]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [219]:
df = df.iloc[1:, 1:]
df = df.rename(columns=dict(zip(df.columns, ['Date', 'Time', 'Weight'])))
df.replace('-', np.nan, inplace=True)

In [220]:
df = df.dropna(how='all')
duplicated_rows = df.duplicated(keep=False)

if duplicated_rows.sum() > 0:
    print(f"Found {duplicated_rows.sum()} duplicated rows.")
    duplicated_sorted = df[duplicated_rows].sort_values(by='Date')
    print(duplicated_sorted)
    df = df.drop_duplicates()
    print(f"Removed duplicates.")

In [221]:
df_google_sheets = df.sort_values(by='Date')

In [222]:
df_google_sheets

,Date,Time,Weight
59,2016-01-03,08:15,76.1
60,2016-01-10,08:20,74.9
61,2016-01-17,08:30,76.1
62,2016-01-24,07:45,75.5
63,2016-01-31,07:15,75.7
...,...,...,...
46,2020-11-01,07:10,"79,3"
47,2020-11-08,07:15,"79,3"
48,2020-11-15,07:55,"78,8"
49,2020-11-22,07:30,"79,5"


In [146]:
print(f'Number of entries: {len(df_google_sheets)}')
print(f'Date of first entry: {df_google_sheets["Date"].iloc[0]}')
print(f'Date of last entry: {df_google_sheets["Date"].iloc[-1]}')

Number of entries: 87
Date of first entry: 2016-01-03
Date of last entry: 2020-11-29


**Garmin Connect**



In [147]:
garmin_connect_masa_ciala = config['garmin_connect_masa_ciala']
file_paths = glob.glob(garmin_connect_masa_ciala)

dfs = []
for path in file_paths:
    df = pd.read_csv(path)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df = df.iloc[:, :2]

In [148]:
data_values = df.values.tolist()
merged_data = [data_values[i] + data_values[i+1] for i in range(0, len(data_values)-1)]
df = pd.DataFrame(merged_data, columns=list(df.columns)*2)

index = df.columns.tolist().index('Czas', 1)
df.columns.values[index] = 'Czas2'
df = df[~df['Czas'].str.contains(':')]

df = df.iloc[:, [0, 2, 3]]

In [149]:
def replace_month(date_str):
    months_pl_to_en = {
        'Sty': 'Jan', 'Lut': 'Feb', 'Mar': 'Mar', 'Kwi': 'Apr',
        'Maj': 'May', 'Cze': 'Jun', 'Lip': 'Jul', 'Sie': 'Aug',
        'Wrz': 'Sep', 'Paź': 'Oct', 'Lis': 'Nov', 'Gru': 'Dec'
    }
    for pl, en in months_pl_to_en.items():
        date_str = date_str.replace(pl, en)
    return date_str.strip()

df_copy = df.copy()
df_copy.loc[:, 'Czas'] = df_copy['Czas'].apply(replace_month)
df_copy.loc[:, 'Czas'] = pd.to_datetime(df_copy['Czas'], format="%d %b %Y").dt.strftime('%Y-%m-%d')
df = df_copy

In [150]:
df = df.rename(columns={'Czas': 'Date', 'Czas2' : 'Time', 'Ciężar': 'Weight'})
df = df.reindex(columns=['Date', 'Time', 'Weight'])
df['Weight'] = df['Weight'].str.replace(' kg', '').astype(float)

In [151]:
df = df.dropna(how='all')
duplicated_rows = df.duplicated(keep=False)

if duplicated_rows.sum() > 0:
    print(f"Found {duplicated_rows.sum()} duplicated rows.")
    duplicated_sorted = df[duplicated_rows].sort_values(by='Date')
    print(duplicated_sorted)
    df = df.drop_duplicates()
    print(f"Removed duplicates.")

Found 12 duplicated rows.
           Date   Time  Weight
732  2024-02-05  06:55    85.2
766  2024-02-05  06:55    85.2
730  2024-02-06  06:55    84.7
764  2024-02-06  06:55    84.7
728  2024-02-12  06:35    84.6
762  2024-02-12  06:35    84.6
726  2024-02-13  06:45    84.6
760  2024-02-13  06:45    84.6
724  2024-02-16  07:35    85.7
758  2024-02-16  07:35    85.7
722  2024-02-19  07:10    84.0
756  2024-02-19  07:10    84.0
Removed duplicates.


In [152]:
df_garmin_connect = df.sort_values(by=['Date', 'Time'], ascending=True)

In [153]:
print(f'Number of entries: {len(df_garmin_connect)}')
print(f'Date of first entry: {df_garmin_connect["Date"].iloc[0]}')
print(f'Date of last entry: {df_garmin_connect["Date"].iloc[-1]}')

Number of entries: 378
Date of first entry: 2020-12-06
Date of last entry: 2024-03-01


**Data Consalidation**

In [154]:
df_merged = pd.concat([df_google_sheets, df_garmin_connect], ignore_index=True)

In [155]:
df_merged = df_merged.dropna(how='all')
duplicated_rows = df_merged.duplicated(keep=False)

if duplicated_rows.sum() > 0:
    print(f"Found {duplicated_rows.sum()} duplicated rows.")
    duplicated_sorted = df[duplicated_rows].sort_values(by='Date')
    print(duplicated_sorted)
    df = df.drop_duplicates()
    print(f"Removed duplicates.")

In [156]:
print(f'Number of entries: {len(df_merged)}')
print(f'Date of first entry: {df_merged["Date"].iloc[0]}')
print(f'Date of last entry: {df_merged["Date"].iloc[-1]}')

Number of entries: 465
Date of first entry: 2016-01-03
Date of last entry: 2024-03-01


In [157]:
# Export the merged dataframe to the weight.csv file
df_merged.to_csv('/content/drive/MyDrive/Colab Notebooks/Tableau/Outputs/weight.csv', index=False)